In [1]:
import keras

In [2]:
from keras.datasets import mnist

> MNIST 데이터를 자동으로 다운로드 해서 데이터를 학습용 화상, 라벨과 평가용 화상, 라벨의 각각으로 분류하고 numpy의 array로 되돌려줌

In [3]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

11493376/11490434 [==============================] - 9s 1us/step


> 화상의 매수 * 세로 화소 수 * 가로 화소 수 * 색(흑백=1) 의 4차원 텐서로 변환

In [11]:
img_rows, img_cols = 28, 28

X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 1)
X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 1)
input_shape = (img_rows, img_cols, 1)

>뉴럴네트워크에서는 입력을 0부터 1과 같은 작은 범위에 두면 무게의 초기값과 학습 계수의 스케일 조정이 불필요함.
>정수형에서 부동소수점형으로 변환한다음 전 데이터를 요소의 최대값(255)로 나누는 조작.

In [12]:
X_train = X_train.astype('float32')/255
X_test = X_test.astype('float32')/255

원핫인코딩에서는 정답이 되는 차원만이 값 1이 되고 나머지는 0이 된다. 뉴럴네트워크의 출력틍이 클래스 수(숫자 0~9)로 이루어지기 때문에 출력층에 대한 교사신호로서 가하기 쉬운 형식으로 변환

>정답라벨은 화상이 나타내는 숫자(0~9)를 원핫인코딩하여 10차원 벡터로 변환

In [6]:
from keras.utils import to_categorical
Y_train = to_categorical(y_train)
Y_test = to_categorical(y_test)

> 합성곱 뉴럴네트워크의 구조 정의 : (합성곱 (3*3), 풀링 (2*2)를 2회 반복하고 그 출력을 1차원으로 바꾸어(Flatten)나열하고나서 2개층의 뉴럴네트워크에 건네 분류하도록 함. 활성화함수('softmax','rm','ReLU')

In [9]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

n_out = len(Y_train[0])

model = Sequential()
model.add(Conv2D(16, kernel_size=(3,3), activation='relu', input_shape = input_shape))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(32, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(n_out, activation='softmax'))
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 16)        160       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 16)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 32)        4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 32)          0         
_________________________________________________________________
flatten (Flatten)            (None, 800)               0         
_________________________________________________________________
dense (Dense)                (None, 128)               102528    
_________________________________________________________________
dense_1 (Dense)              (None, 10)               

> compile메소드로 평가함수(categorical_crossentropy)와 최적화기(rmsprop)를 가하고 나서 fit메소드로 학습 수행

In [10]:
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
model.fit(X_train, Y_train, epochs=5, batch_size=200)
score = model.evaluate(X_test, Y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Epoch 1/5
300/300 [==============================] - 31s 105ms/step - loss: 0.2949 - accuracy: 0.9136
Epoch 2/5
300/300 [==============================] - 32s 106ms/step - loss: 0.0753 - accuracy: 0.9773
Epoch 3/5
300/300 [==============================] - 32s 105ms/step - loss: 0.0507 - accuracy: 0.9844
Epoch 4/5
300/300 [==============================] - 32s 108ms/step - loss: 0.0380 - accuracy: 0.9886
Epoch 5/5
300/300 [==============================] - 33s 108ms/step - loss: 0.0306 - accuracy: 0.9906
Test loss: 0.03312717005610466
Test accuracy: 0.9887999892234802
